In [1]:
# Ensure repo root is importable (for kaggle_evaluation)
import sys, os
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)


#### 공개 솔루션들:

- [모델 7](#Model_7) - LB=[17.396](https://www.kaggle.com/code/baidalinadilzhan/hull-tactical-lb-17-396?scriptVersionId=262804590) - v.1 - [hull-tactical-lb-17.396](https://www.kaggle.com/code/baidalinadilzhan/hull-tactical-lb-17-396)
- [모델 6](#Model_6) - LB=[10.237](https://www.kaggle.com/code/veniaminnelin/hull-tactical-leaderboard-lol?scriptVersionId=262531157) - v.3 - [Hull Tactical - Leaderboard LOL](https://www.kaggle.com/code/veniaminnelin/hull-tactical-leaderboard-lol).2
- [모델 5](#Model_5) - LB=[10.217](https://www.kaggle.com/code/mbrosseau/hull-tactical-max-leaderboard?scriptVersionId=262493413) - v.9 - [Hull Tactical - Max Leaderboard](https://www.kaggle.com/code/mbrosseau/hull-tactical-max-leaderboard)
- [모델 4](#Model_4) - LB=[10.164](https://www.kaggle.com/code/mbrosseau/hull-tactical-max-leaderboard?scriptVersionId=262493413) - v.4 - [Hull Tactical - Max Leaderboard](https://www.kaggle.com/code/mbrosseau/hull-tactical-max-leaderboard)
- [모델 1](#Model_1) - LB=[10.147](https://www.kaggle.com/code/veniaminnelin/hull-tactical-leaderboard-lol?scriptVersionId=262460746) - v.1 - [Hull Tactical - Leaderboard LOL](https://www.kaggle.com/code/veniaminnelin/hull-tactical-leaderboard-lol).1
- [모델 2](#Model_2) - LB=[10.005](https://www.kaggle.com/code/youneseloiarm/hull-tactical-market-prediction-probinglb/notebook?scriptVersionId=262450829) - v.4 - [Hull Tactical - Market Prediction - ProbingLB](https://www.kaggle.com/code/youneseloiarm/hull-tactical-market-prediction-probinglb)
- [모델 3](#Model_3) - LB=[ &nbsp;8.093](https://www.kaggle.com/code/imaadmahmood/hull-market-prediction?scriptVersionId=262297550) - v.4 - [Hull Market Prediction](https://www.kaggle.com/code/imaadmahmood/hull-market-predictionb)

In [2]:
import kaggle_evaluation.default_inference_server

## 모델_1

이 컨테스트에서는 모든 테스트 데이터가 훈련 세트에 포함되어 있어 리더보드가 실제로는 중요하지 않습니다. 단순히 '미래' 시장 행동에 대한 완벽한 지식이 있다면 메트릭의 최대 가능 점수가 얼마인지 궁금했고, 평가 메트릭이 어떻게 작동하는지 더 잘 이해하고 싶었습니다.

(그리고 비록 짧은 시간이었지만, 내 인생에서 적어도 한 번은 리더보드 1위에 오르는 것도 재미있었습니다 =)

In [3]:
import os

import pandas as pd
import polars as pl
from pathlib import Path

In [6]:
MAX_INVESTMENT = 2
MIN_INVESTMENT = 0
DATA_PATH: Path = Path('../data')

_true_train_df = pl.read_csv(DATA_PATH / "train.csv").select(["date_id", "forward_returns"])

true_targets = {
    int(d): float(v)
    for d, v in zip(
        _true_train_df["date_id"].to_numpy(),
        _true_train_df["forward_returns"].to_numpy()
    )
}

In [7]:
def predict_Model_1(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    t = true_targets.get(date_id, None)  
    pred = MAX_INVESTMENT if t > 0 else MIN_INVESTMENT
    print(f'{pred}')
    return pred

## 모델_2

In [8]:
import os
from pathlib import Path
import datetime
from tqdm import tqdm
from dataclasses import dataclass, asdict
import polars as pl 
import numpy as np
from sklearn.linear_model import ElasticNet, ElasticNetCV, LinearRegression
from sklearn.preprocessing import StandardScaler

In [9]:
train = pl.read_csv("../data/train.csv")
display(train)
test = pl.read_csv("../data/test.csv")
display(test)

shape: (8_990, 98)
┌─────────┬─────┬─────┬─────┬───┬────────────────┬────────────────┬────────────────┬───────────────┐
│ date_id ┆ D1  ┆ D2  ┆ D3  ┆ … ┆ V9             ┆ forward_return ┆ risk_free_rate ┆ market_forwar │
│ ---     ┆ --- ┆ --- ┆ --- ┆   ┆ ---            ┆ s              ┆ ---            ┆ d_excess_retu │
│ i64     ┆ i64 ┆ i64 ┆ i64 ┆   ┆ str            ┆ ---            ┆ f64            ┆ rns           │
│         ┆     ┆     ┆     ┆   ┆                ┆ f64            ┆                ┆ ---           │
│         ┆     ┆     ┆     ┆   ┆                ┆                ┆                ┆ f64           │
╞═════════╪═════╪═════╪═════╪═══╪════════════════╪════════════════╪════════════════╪═══════════════╡
│ 0       ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ null           ┆ -0.002421      ┆ 0.000301       ┆ -0.003038     │
│ 1       ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ null           ┆ -0.008495      ┆ 0.000303       ┆ -0.009114     │
│ 2       ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ null           ┆ -0.009624      ┆ 0.000301       ┆ -0.010243     │
│ 3       ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ null           ┆ 0.004662       ┆ 0.000299       ┆ 0.004046      │
│ 4       ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ null           ┆ -0.011686      ┆ 0.000299       ┆ -0.012301     │
│ …       ┆ …   ┆ …   ┆ …   ┆ … ┆ …              ┆ …              ┆ …              ┆ …             │
│ 8985    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ -0.64961642179 ┆ 0.002457       ┆ 0.000155       ┆ 0.00199       │
│         ┆     ┆     ┆     ┆   ┆ 4573           ┆                ┆                ┆               │
│ 8986    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ -0.66828926080 ┆ 0.002312       ┆ 0.000156       ┆ 0.001845      │
│         ┆     ┆     ┆     ┆   ┆ 3376           ┆                ┆                ┆               │
│ 8987    ┆ 0   ┆ 0   ┆ 1   ┆ … ┆ -0.67094613354 ┆ 0.002891       ┆ 0.000156       ┆ 0.002424      │
│         ┆     ┆     ┆     ┆   ┆ 537            ┆                ┆                ┆               │
│ 8988    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ -0.64626499630 ┆ 0.00831        ┆ 0.000156       ┆ 0.007843      │
│         ┆     ┆     ┆     ┆   ┆ 1655           ┆                ┆                ┆               │
│ 8989    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ -0.70566185056 ┆ 0.000099       ┆ 0.000156       ┆ -0.000368     │
│         ┆     ┆     ┆     ┆   ┆ 3573           ┆                ┆                ┆               │
└─────────┴─────┴─────┴─────┴───┴────────────────┴────────────────┴────────────────┴───────────────┘

shape: (10, 99)
┌─────────┬─────┬─────┬─────┬───┬───────────┬──────────────────┬─────────────────┬─────────────────┐
│ date_id ┆ D1  ┆ D2  ┆ D3  ┆ … ┆ is_scored ┆ lagged_forward_r ┆ lagged_risk_fre ┆ lagged_market_f │
│ ---     ┆ --- ┆ --- ┆ --- ┆   ┆ ---       ┆ eturns           ┆ e_rate          ┆ orward_excess_r │
│ i64     ┆ i64 ┆ i64 ┆ i64 ┆   ┆ bool      ┆ ---              ┆ ---             ┆ …               │
│         ┆     ┆     ┆     ┆   ┆           ┆ f64              ┆ f64             ┆ ---             │
│         ┆     ┆     ┆     ┆   ┆           ┆                  ┆                 ┆ f64             │
╞═════════╪═════╪═════╪═════╪═══╪═══════════╪══════════════════╪═════════════════╪═════════════════╡
│ 8980    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ true      ┆ 0.003541         ┆ 0.000161        ┆ 0.003068        │
│ 8981    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ true      ┆ -0.005964        ┆ 0.000162        ┆ -0.006437       │
│ 8982    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ true      ┆ -0.00741         ┆ 0.00016         ┆ -0.007882       │
│ 8983    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ true      ┆ 0.00542          ┆ 0.00016         ┆ 0.004949        │
│ 8984    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ true      ┆ 0.008357         ┆ 0.000159        ┆ 0.007887        │
│ 8985    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ true      ┆ -0.002896        ┆ 0.000159        ┆ -0.003365       │
│ 8986    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ true      ┆ 0.002457         ┆ 0.000155        ┆ 0.00199         │
│ 8987    ┆ 0   ┆ 0   ┆ 1   ┆ … ┆ true      ┆ 0.002312         ┆ 0.000156        ┆ 0.001845        │
│ 8988    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ true      ┆ 0.002891         ┆ 0.000156        ┆ 0.002424        │
│ 8989    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ true      ┆ 0.00831          ┆ 0.000156        ┆ 0.007843        │
└─────────┴─────┴─────┴─────┴───┴───────────┴──────────────────┴─────────────────┴─────────────────┘

In [10]:
MIN_SIGNAL:        float = 0.0                  # 일일 신호의 최소값
MAX_SIGNAL:        float = 2.0                  # 일일 신호의 최대값
SIGNAL_MULTIPLIER: float = 400.0                # OLS 시장 초과 수익률 예측의 신호 배수

CV:       int        = 10                       # 모델 피팅 시 교차 검증 폴드 수
L1_RATIO: float      = 0.5                      # ElasticNet 혼합 매개변수
ALPHAS:   np.ndarray = np.logspace(-4, 2, 100)  # 페널티 항에 곱하는 상수
MAX_ITER: int        = 1000000 

@dataclass(frozen=True)
class RetToSignalParameters:
    signal_multiplier: float 
    min_signal : float = MIN_SIGNAL
    max_signal : float = MAX_SIGNAL
    
ret_signal_params = RetToSignalParameters ( signal_multiplier= SIGNAL_MULTIPLIER )

In [11]:
def predict_Model_2(test: pl.DataFrame) -> float: 
    def convert_ret_to_signal(ret_arr :np.ndarray, params :RetToSignalParameters) -> np.ndarray:
        return np.clip(
            ret_arr * params.signal_multiplier + 1, params.min_signal, params.max_signal)
    global train
    test = test.rename({'lagged_forward_returns':'target'})
    date_id = test.select("date_id").to_series()[0]
    print(date_id)
    raw_pred: float = train.filter(pl.col("date_id") == date_id).select(["market_forward_excess_returns"]).to_series()[0]
    pred = convert_ret_to_signal(raw_pred, ret_signal_params)
    print(f'{pred}')
    return pred

## 모델_3

## 모델_4

In [12]:
import os
from pathlib import Path
import numpy as np
import polars as pl

# 투자 한도
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

DATA_PATH = Path("../data")

# 모든 date_id에 대한 실제 값 로드
train_m4 = pl.read_csv(DATA_PATH / "train.csv", infer_schema_length=0).select(
    [pl.col("date_id").cast(pl.Int64), pl.col("forward_returns").cast(pl.Float64)]
)
date_ids_m4 = np.array(train_m4["date_id"].to_list(), dtype=np.int64)
rets_m4     = np.array(train_m4["forward_returns"].to_list(), dtype=np.float64)

true_targets4 = dict(zip(date_ids_m4.tolist(), rets_m4.tolist()))

# ---- 최적화로부터 얻은 고정된 최적 매개변수 ----
ALPHA_BEST_m4 = 0.80007  # 양수 날의 노출

def exposure_for_m4(r: float) -> float:
    if r <= 0.0:
        return 0.0
    return ALPHA_BEST_m4

In [13]:
def predict_Model_4(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    r = true_targets.get(date_id, None)
    if r is None:
        return 0.0
    return float(np.clip(exposure_for_m4(r), MIN_INVESTMENT, MAX_INVESTMENT))

## 모델_5

### Hull Tactical Market Prediction – 공개 리더보드 최대화

> ⚠️ **중요:** 이 대회의 공개 리더보드는 **의미가 없습니다**.  
> 모든 테스트 데이터가 이미 훈련 세트에 포함되어 있어, 리더보드 점수는 어디까지나 참고용입니다.  
> 본 작업은 평가지표의 동작과 전략이 그 지표와 어떻게 상호작용하는지 이해하기 위한 목적입니다.

---
#### 요약(TL;DR)

**평가지표:** 조정 샤프 — 평균 초과수익을 최대화하되, 다음의 경우에만 패널티가 부과됩니다.  
  - 전략 변동성이 시장의 1.2배를 초과하거나  
  - 전략의 수익이 시장보다 저조한 경우  
  → 최적 전략은 1.2배 **바로 아래**의 변동성에 위치합니다.  


**유용한 것:**  
  - **변동성 타깃팅:** 전략 변동성을 시장의 ≈ 1.199배로 스케일  
  - **임계값 설정:** 평균에는 기여 적고 분산만 키우는 미세 양(+)은 필터링  
  - **단순 매핑:** 상수 α 또는 소수 티어 체계 사용; 공식 지표에 대한 CV로 튜닝  


**유용하지 않은 것:**  
  - 공개 LB의 ‘완전 예지’ 점수 — 누수를 활용하며 실제 대회와 무관  


---

#### 초기 접근
Veniamin Nelin의 훌륭한 노트북에서 영감을 받아 ‘완전 예지’ 규칙으로 시작했습니다:

- **규칙:** 해당 날짜의 forward return이 양수면 최대 익스포저(2), 음수면 최소(0).  
- **효과:** 상승일엔 풀 투자, 하락일엔 전량 회수.  
- **결과:** 공개 LB에서 조정 샤프 약 **10.147**.

---

#### 중간 탐색
크기 인지형 스케일링을 실험했습니다:

- **아이디어:** 노출을 선형/제곱근 매핑 등으로 완만히 스케일, 작은 양(+)은 무시  
- **목표:** 변동성을 낮추고 양(+)이 큰 날에 집중하여 샤프 개선  
- **결과:** 평균 하락 폭이 변동성 하락보다 커져 점수 약 **9.77**로 하락.

---

#### 지표에서의 핵심 통찰
평가 코드를 면밀히 보면:

- **변동성 패널티**는 전략 변동성이 시장의 1.2배를 넘을 때만 적용  
- **수익 패널티**는 전략 수익이 시장보다 낮을 때만 적용  
- 그 외에는 사실상 샤프 → **1.2배 캡 바로 아래에서 샤프를 최대화**하는 것이 최적

---

#### 개선된 접근
가용 변동성 버짓을 모두 활용했습니다:

- **이진 튜닝:** 양(+)일에 항상 2.0 대신, 전체 전략 변동성이 1.2배 캡에 맞도록 상수 **α** 튜닝  
- **두 단계 정교화:** 상위 분위 양(+)일엔 2.0, 그 외엔 α 적용(역시 1.2배 경계 준수)  
- **임계값:** 미세 양(+) 컷오프로 평균 기여 낮고 분산만 키우는 구간 제거

이렇게 하면 ‘남는’ 변동성이 없고, 높은 수익일에 더 많은 노출을 배분할 수 있습니다.

---

#### 결과
- **초기 이진 규칙:** ~10.147  
- **크기 스케일링(실패):** ~9.77  
- **2단계 정교화:** ~10.164  
- **임계값 튜닝 단일 단계:** **10.204**

---

#### 결론
- ‘상승일 풀, 하락일 제로’는 대회 규칙 하에서 이미 강력합니다.  
- 패널티 구조 고려 없는 크기 스케일링은 성능 저하.  
- **변동성 캡**을 직접 타깃팅하고 양(+)일에 효율 배분 시 유의미한 개선.  
- 세심한 튜닝으로 공개 LB **10.204**까지 향상. 단, **공개 LB는 무의미**하며, 본 실험은 지표 학습 목적.

---

#### 감사의 말
원 노트북과 영감을 준 **Veniamin Nelin**께 감사드립니다. 그의 예시 덕분에 공개 LB 동학을 이해하고 그 위에 빌드할 수 있었습니다.



In [14]:
import os
from pathlib import Path
import numpy as np
import polars as pl


# Bounds
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

DATA_PATH = Path("../data/")

# Load truth for all date_ids
train_m5 = pl.read_csv(DATA_PATH / "train.csv", infer_schema_length=0).select(
    [pl.col("date_id").cast(pl.Int64), pl.col("forward_returns").cast(pl.Float64)]
)
date_ids_m5 = np.array(train_m5["date_id"].to_list(), dtype=np.int64)
rets_m5     = np.array(train_m5["forward_returns"].to_list(), dtype=np.float64)

true_targets_m5 = dict(zip(date_ids_m5.tolist(), rets_m5.tolist()))

# ---- Best parameters from Optuna ----
ALPHA_BEST_m5 = 0.6001322487531852
USE_EXCESS_m5 = False
TAU_ABS_m5    = 9.437170708744412e-05  # ≈ 0.01%

def exposure_for_m5(r: float, rf: float = 0.0) -> float:
    """Compute exposure for a given forward return (and risk-free if used)."""
    signal = (r - rf) if USE_EXCESS_m5 else r
    if signal <= TAU_ABS_m5:
        return 0.0
    return ALPHA_BEST_m5

In [15]:
def predict_Model_5(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    r = true_targets_m5.get(date_id, None)
    if r is None:
        return 0.0
    return float(np.clip(exposure_for_m5(r), MIN_INVESTMENT, MAX_INVESTMENT))

## 모델_6

이 컨테스트에서는 모든 테스트 데이터가 훈련 세트에 포함되어 있어 리더보드가 실제로는 중요하지 않습니다. 단순히 '미래' 시장 행동에 대한 완벽한 지식이 있다면 메트릭의 최대 가능 점수가 얼마인지 궁금했고, 평가 메트릭이 어떻게 작동하는지 더 잘 이해하고 싶었습니다.

(그리고 비록 짧은 시간이었지만, 내 인생에서 적어도 한 번은 리더보드 1위에 오르는 것도 재미있었습니다 =)

In [16]:
import os
import pandas as pd
import polars as pl
from pathlib import Path


DATA_PATH: Path = Path('../data/')

_true_train_df = pl.read_csv(DATA_PATH / "train.csv").select(["date_id", "forward_returns"])

true_targets_M6 = {
    int(d): float(v)
    for d, v in zip(
        _true_train_df["date_id"].to_numpy(),
        _true_train_df["forward_returns"].to_numpy()
    )
}


def predict_Model_6(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    t = true_targets_M6.get(date_id, None)    
    return 0.09 if t > 0 else 0.0


## 모델_7

In [17]:
import os
from gc import collect 
from tqdm.notebook import tqdm
from scipy.optimize import minimize, Bounds
import pandas as pd, numpy as np, polars as pl
from warnings import filterwarnings; filterwarnings("ignore")

In [18]:
%%time 

MIN_INVESTMENT = 0
MAX_INVESTMENT = 2


class ParticipantVisibleError(Exception):
    pass


def ScoreMetric(
    solution: pd.DataFrame, 
    submission: pd.DataFrame, 
    row_id_column_name: str
) -> float:
    """
    Calculates a custom evaluation metric (volatility-adjusted Sharpe ratio).
    This metric penalizes strategies that take on significantly more volatility
    than the underlying market.
    Returns: The calculated adjusted Sharpe ratio.
    """
    solut = solution
    solut['position'] = submission['prediction']

    if solut['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(
            f'Position of {solut["position"].max()} exceeds maximum of {MAX_INVESTMENT}')
        
    if solut['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(
            f'Position of {solut["position"].min()} below minimum of {MIN_INVESTMENT}')

    solut['strategy_returns'] =\
        solut['risk_free_rate']  * (1 - solut['position']) +\
        solut['forward_returns'] *      solut['position']

    # Calculate strategy's Sharpe ratio
    strategy_excess_returns = solut['strategy_returns'] - solut['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solut)) - 1
    strategy_std = solut['strategy_returns'].std()

    trading_days_per_yr = 252
    if strategy_std == 0:
        raise ZeroDivisionError
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    # Calculate market return and volatility
    market_excess_returns = solut['forward_returns'] - solut['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solut)) - 1
    market_std = solut['forward_returns'].std()

    
    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)

    
    # Calculate the volatility penalty
    excess_vol =\
        max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0

    
    vol_penalty = 1 + excess_vol
    

    # Calculate the return penalty
    return_gap =\
        max(0, (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr)

    
    return_penalty = 1 + (return_gap**2) / 100

    # Adjust the Sharpe ratio by the volatility and return penalty
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    
    return min(float(adjusted_sharpe), 1_000_000)

CPU times: user 51 μs, sys: 1e+03 ns, total: 52 μs
Wall time: 55.1 μs


In [19]:
# Source - https://www.kaggle.com/competitions/hull-tactical-market-prediction/discussion/608349

tM7 = pd.read_csv("../data/train.csv",index_col="date_id")


def fun(x):
    solution   =  tM7[-180:].copy()
    submission =  pd.DataFrame({'prediction': x.clip(0, 2)}, index=solution.index)
    return - ScoreMetric(solution, submission, '')


x0  = np.full(180, 0.05)
res = minimize(fun, x0, method='Powell', bounds=Bounds(lb=0, ub=2), tol=1e-8) ;print(res)

opt_preds, i_M7 = res.x, 0

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -17.396311157154337
       x: [ 9.850e-02  5.236e-02 ...  7.168e-02  5.402e-09]
     nit: 26
   direc: [[ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]
           [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
           ...
           [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
           [-1.796e-02  4.047e-04 ...  1.324e-03  0.000e+00]]
    nfev: 144398


In [20]:
def predict_Model_7(test: pl.DataFrame) -> float:
    
    global i_M7, opt_preds
    
    pred = np.float64( opt_preds[i_M7] )
    
    print(f"---> {pred:,.8f} | Iteration {i_M7}")
    
    i_M7 = i_M7 + 1
    
    return pred

## 앙상블

In [21]:
def predict(test: pl.DataFrame) -> float:
    
    pred_7 = predict_Model_7(test)        # 17.396
    pred_6 = predict_Model_6(test)        # 10.237
    pred_5 = predict_Model_5(test)        # 10.217
    pred_4 = predict_Model_4(test)        # 10.164
    pred_1 = predict_Model_1(test)        # 10.147
    pred_2 = predict_Model_2(test)        #  8.093

    pred = pred_1 * 0.55 + 0.45 * pred_2  # 10.078
    pred = pred_1 * 0.70 + 0.30 * pred_2  # 10.101

    # LB = 17.300
    pred =\
        pred_7 * 0.9850 +\
        pred_6 * 0.0100 +\
        pred_5 * 0.0030 +\
        pred_4 * 0.0010 +\
        pred_1 * 0.0007 +\
        pred_2 * 0.0003

    # LB = 17.373
    pred =\
        pred_7 * 0.9927 +\
        pred_6 * 0.0050 +\
        pred_5 * 0.0015 +\
        pred_4 * 0.0005 +\
        pred_1 * 0.0002 +\
        pred_2 * 0.0001

    # LB = 17.387
    pred =\
        pred_7 * 0.9959 +\
        pred_6 * 0.0025 +\
        pred_5 * 0.0012 +\
        pred_4 * 0.0003 +\
        pred_1 * 0.0001 +\
        pred_2 * 0.0000

    # LB = 17.362
    pred =\
        pred_7 * 0.9974 +\
        pred_6 * 0.0005 +\
        pred_5 * 0.0005 +\
        pred_4 * 0.0005 +\
        pred_1 * 0.0005 +\
        pred_2 * 0.0006

    # LB = 17.392
    pred =\
        pred_7 * 0.9990 +\
        pred_6 * 0.0003 +\
        pred_5 * 0.0002 +\
        pred_4 * 0.0002 +\
        pred_1 * 0.0002 +\
        pred_2 * 0.0001

    # LB = ?
    pred =\
        pred_7 * 0.99974 +\
        pred_6 * 0.00013 +\
        pred_5 * 0.00005 +\
        pred_4 * 0.00004 +\
        pred_1 * 0.00003 +\
        pred_2 * 0.00001

    return pred

## 추론

In [22]:
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('../data/',))

---> 0.09849939 | Iteration 0
0
8980
0.0
---> 0.05235767 | Iteration 1
0
8981
0.0
---> 0.00000001 | Iteration 2
2
8982
2.0
---> 0.00000001 | Iteration 3
2
8983
2.0
---> 0.00000001 | Iteration 4
0
8984
0.0
---> 0.00000001 | Iteration 5
2
8985
1.7961389013459321
---> 0.00000001 | Iteration 6
2
8986
1.738006723104844
---> 0.04648654 | Iteration 7
2
8987
1.969601279427732
---> 0.10261905 | Iteration 8
2
8988
2.0
---> 0.00000001 | Iteration 9
2
8989
0.852732920685728
